In [1]:
from FM_solution import *

## 가상의 주식 테이블 생성하기
- 시점 $T$ 는 1부터 1000까지 생성
- 개별 종목의 갯수는 400개로 설정
- 난수 발생으로 임의로 정하였기 때문에, 데이터 포맷 그 이상의 의미를 가지지 못한다.

In [2]:
code_df = pd.read_csv('industrial_code.csv')
code_df.rename(columns={'종목코드':'code'},inplace=True)
code_df = code_df[:401]
num_of_stock = 400
T = 1000
df = pd.DataFrame(columns=[i for i in range(1,T+1)])

for i in df.columns:
    df[i] = [0 for i in range(num_of_stock + 1)]
    
df.index = list(code_df['code'])
df.index = [str(i) for i in df.index]

for i in range(len(df)):
    df.iloc[i] = [(np.random.randint(10))*(1+np.random.rand()) for _ in range(1000)]
    
df.index = [str(i) for i in df.index]
df = df.append(pd.Series([1 for _ in range(T)], index=df.columns), ignore_index=True) 
df.rename(index={df.index[-1]:'bias'},inplace=True)
df.rename(index={df.index[-2]:'market'},inplace=True)
individual_stock_df = df[(df.index != 'market') & (df.index != 'bias')]
df.index = list(code_df['code'])[:-1] + ['market','bias']
display(df.tail())
print(df.shape)

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
17810,5.966439,14.511612,10.210677,6.147145,10.969940,15.778136,4.715378,16.039868,4.357659,11.686800,...,2.648514,1.057011,10.716836,13.566762,5.916202,1.918234,7.225932,3.365132,8.955421,5.809034
11200,3.134063,4.841810,0.000000,8.764466,9.499532,9.339222,1.906553,8.173353,2.618946,0.000000,...,5.625720,7.534312,10.985695,11.641516,8.836607,6.103713,0.000000,5.592094,9.597813,14.188030
17390,10.144767,13.074544,11.308739,10.238510,5.463524,4.663494,5.290853,4.977656,7.104612,3.990398,...,8.519255,5.209785,15.293522,11.508067,5.893690,8.340524,11.119110,5.394857,1.183558,13.917283
market,13.978080,0.000000,11.944979,2.374868,17.617177,7.224437,13.038061,7.316876,1.221582,5.461273,...,5.861701,9.034882,12.803319,12.179560,6.619743,8.168163,4.058974,8.682541,1.669125,1.986219
bias,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(402, 1000)


## 1단계 : 포트폴리오 구성시기 (portfolio formation period)
- 시장모형의 시계열회귀분석으로 개별 종목에 대한 $\beta$ 계수를 추정한다.
- 개별 종목에 대한 $\beta$ 400개 계수 측정이 완료되면, 크기 순으로 나열해서 20개씩 묶어, 포트폴리오를 형성한다.
    - 개별 종목의 갯수를 400개로 한 이유가 이와 같다.

In [3]:
testing_beta , testing_resid = first_OLS(individual_stock_df,df)

In [4]:
len(testing_beta) , len(testing_resid)

(400, 400)

In [5]:
beta , resid , stock , idx , testing_df = first_sorting_beta_resid(testing_beta,testing_resid,df)

In [6]:
len(beta) , len(resid) , len(stock) , len(idx)

(20, 20, 20, 20)

In [7]:
testing_df.tail()

,beta,resid,valued_stock_idx,valued_stock
15,0.675039,5.509134,"[149, 233, 332, 14, 74, 161, 68, 258, 244, 31,...","[105840, 90430, 6840, 285130, 112610, 119650, ..."
16,0.669721,5.569581,"[23, 157, 305, 37, 195, 279, 389, 69, 148, 241...","[271980, 31440, 37710, 241560, 32640, 72710, 1..."
17,0.663933,5.458809,"[326, 232, 11, 45, 49, 273, 56, 182, 119, 382,...","[7120, 90350, 18250, 16740, 195870, 24110, 755..."
18,0.656410,5.492643,"[58, 84, 99, 130, 21, 356, 396, 140, 201, 83, ...","[89590, 185750, 161390, 20150, 55490, 25530, 1..."
19,0.638963,5.622237,"[276, 304, 330, 131, 213, 43, 384, 116, 66, 12...","[7660, 64960, 9540, 11210, 97230, 249420, 1658..."


# 2단계 : 포트폴리오 $\beta$ 의 추정시기 (estimation period)

- 시장 모형의 시계열회귀분석으로 20개 포트폴리오에 대한 $\beta$ 계수를 추정한다.
- 개별 종목에 대한 $\beta_i$ 를 구하고, 포트폴리오를 형성, 시계열 평균으로 $\beta_P$ 를 추정한다.

In [8]:
testing_beta2,testing_resid2 = second_OLS(individual_stock_df,df)

In [9]:
len(testing_beta2) , len(testing_resid2)

(400, 400)

In [10]:
final_beta , final_resid , final_df = second_sorting_beta_resid(testing_beta2,testing_resid2,df,testing_df)

In [11]:
len(final_beta) , len(final_resid)

(20, 20)

In [12]:
final_df.tail()

,beta,resid,stock_idx,stock
15,0.696485,5.589463,"[149, 233, 332, 14, 74, 161, 68, 258, 244, 31,...","[105840, 90430, 6840, 285130, 112610, 119650, ..."
16,0.679603,5.554716,"[23, 157, 305, 37, 195, 279, 389, 69, 148, 241...","[271980, 31440, 37710, 241560, 32640, 72710, 1..."
17,0.687030,5.597519,"[326, 232, 11, 45, 49, 273, 56, 182, 119, 382,...","[7120, 90350, 18250, 16740, 195870, 24110, 755..."
18,0.680400,5.502857,"[58, 84, 99, 130, 21, 356, 396, 140, 201, 83, ...","[89590, 185750, 161390, 20150, 55490, 25530, 1..."
19,0.685451,5.572138,"[276, 304, 330, 131, 213, 43, 384, 116, 66, 12...","[7660, 64960, 9540, 11210, 97230, 249420, 1658..."


# 3단계 : 검증시기(testing period)
- 앞의 2단계에 의한여 20개의 포트폴리오의 베타 $\beta_P$ 가 추정될 것이므로 이 $\beta_P$ 와 포트폴리오에 속한 개별증권의 1개월간의 기대수익률과의 횡단면 회귀분석을 하여 식의 회귀 계수를 각각 추정한다.

In [13]:
gamma_1 , gamma_2 , gamma_3 , gamma_4 = real_df(individual_stock_df,final_df)

In [14]:
gamma_1 , gamma_2 , gamma_3 , gamma_4

([-9494.2888070186018],
 [27653.742395225563],
 [-20122.062117695241],
 [0.050335446749860679])